In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv

load_dotenv()

False

In [4]:
KEY = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(
    openai_api_key = KEY,
    model_name = "gpt-3.5-turbo",
    temperature = 0.5,
)

In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions fro {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be confirming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [10]:
quiz_chain = LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key="quiz",
    verbose=True,
)

In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject, quiz"],
    template=TEMPLATE
)

In [13]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_generation_prompt,
    output_key="review",
    verbose=True,
)

In [14]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [15]:
file_path = "../data.txt"

In [16]:
with open(file_path, 'r') as f:
    TEXT = f.read()

In [17]:
# serialize python dictionary into a JSON-formatted string

json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [18]:
NUMBER = 5
SUBJECT = "RAG using Llama Index"
TONE = "simple"

In [19]:
# How to setup token usage tracking in langchain

with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON),
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Retrieval Augmented Generation (RAG)
LLMs are trained on enormous bodies of data but they aren’t trained on your data. Retrieval-Augmented Generation (RAG) solves this problem by adding your data to the data LLMs already have access to. You will see references to RAG frequently in this documentation.

In RAG, your data is loaded and prepared for queries or “indexed”. User queries act on the index, which filters your data down to the most relevant context. This context and your query then go to the LLM along with a prompt, and the LLM provides a response.

Even if what you’re building is a chatbot or an agent, you’ll want to know RAG techniques for getting data into your application.



Stages within RAG
There are five key stages within RAG, which in turn will be a part of any larger application you build. These are:

Loading: this refers to getting your data from where it lives –

In [20]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:3235
Prompt Tokens:2460
Completion Tokens:775
Total Cost:0.00524


In [21]:
response

{'text': 'Retrieval Augmented Generation (RAG)\nLLMs are trained on enormous bodies of data but they aren’t trained on your data. Retrieval-Augmented Generation (RAG) solves this problem by adding your data to the data LLMs already have access to. You will see references to RAG frequently in this documentation.\n\nIn RAG, your data is loaded and prepared for queries or “indexed”. User queries act on the index, which filters your data down to the most relevant context. This context and your query then go to the LLM along with a prompt, and the LLM provides a response.\n\nEven if what you’re building is a chatbot or an agent, you’ll want to know RAG techniques for getting data into your application.\n\n\n\nStages within RAG\nThere are five key stages within RAG, which in turn will be a part of any larger application you build. These are:\n\nLoading: this refers to getting your data from where it lives – whether it’s text files, PDFs, another website, a database, or an API – into your pip

In [32]:
quiz = response.get("quiz")

In [33]:
quiz

'{"1": {"mcq": "What is the purpose of Retrieval-Augmented Generation (RAG)?", "options": {"a": "To train LLMs on enormous bodies of data", "b": "To add user data to the data LLMs already have access to", "c": "To create vector embeddings for data indexing", "d": "To evaluate the effectiveness of LLM responses"}, "correct": "b"}, \n"2": {"mcq": "Which stage of RAG involves creating a data structure for querying the data?", "options": {"a": "Loading", "b": "Indexing", "c": "Storing", "d": "Querying"}, "correct": "b"}, \n"3": {"mcq": "What is the purpose of embeddings in RAG?", "options": {"a": "To generate numerical representations of data", "b": "To store metadata about the data", "c": "To efficiently retrieve relevant context from an index", "d": "To apply transformations to retrieved nodes"}, "correct": "a"}, \n"4": {"mcq": "What is the role of a router in RAG?", "options": {"a": "To generate a response from an LLM", "b": "To select the best retriever based on metadata and query", "c

In [40]:
quiz

{'1': {'mcq': 'What is the purpose of Retrieval-Augmented Generation (RAG)?',
  'options': {'a': 'To train LLMs on enormous bodies of data',
   'b': 'To add user data to the data LLMs already have access to',
   'c': 'To create vector embeddings for data indexing',
   'd': 'To evaluate the effectiveness of LLM responses'},
  'correct': 'b'},
 '2': {'mcq': 'Which stage of RAG involves creating a data structure for querying the data?',
  'options': {'a': 'Loading',
   'b': 'Indexing',
   'c': 'Storing',
   'd': 'Querying'},
  'correct': 'b'},
 '3': {'mcq': 'What is the purpose of embeddings in RAG?',
  'options': {'a': 'To generate numerical representations of data',
   'b': 'To store metadata about the data',
   'c': 'To efficiently retrieve relevant context from an index',
   'd': 'To apply transformations to retrieved nodes'},
  'correct': 'a'},
 '4': {'mcq': 'What is the role of a router in RAG?',
  'options': {'a': 'To generate a response from an LLM',
   'b': 'To select the best re

In [42]:
qdict = json.loads(json.dumps(quiz))

In [43]:
type(qdict)

dict

In [44]:
qdict.items()

dict_items([('1', {'mcq': 'What is the purpose of Retrieval-Augmented Generation (RAG)?', 'options': {'a': 'To train LLMs on enormous bodies of data', 'b': 'To add user data to the data LLMs already have access to', 'c': 'To create vector embeddings for data indexing', 'd': 'To evaluate the effectiveness of LLM responses'}, 'correct': 'b'}), ('2', {'mcq': 'Which stage of RAG involves creating a data structure for querying the data?', 'options': {'a': 'Loading', 'b': 'Indexing', 'c': 'Storing', 'd': 'Querying'}, 'correct': 'b'}), ('3', {'mcq': 'What is the purpose of embeddings in RAG?', 'options': {'a': 'To generate numerical representations of data', 'b': 'To store metadata about the data', 'c': 'To efficiently retrieve relevant context from an index', 'd': 'To apply transformations to retrieved nodes'}, 'correct': 'a'}), ('4', {'mcq': 'What is the role of a router in RAG?', 'options': {'a': 'To generate a response from an LLM', 'b': 'To select the best retriever based on metadata and

In [34]:
quiz = json.loads(quiz)

In [39]:
quiz

{'1': {'mcq': 'What is the purpose of Retrieval-Augmented Generation (RAG)?',
  'options': {'a': 'To train LLMs on enormous bodies of data',
   'b': 'To add user data to the data LLMs already have access to',
   'c': 'To create vector embeddings for data indexing',
   'd': 'To evaluate the effectiveness of LLM responses'},
  'correct': 'b'},
 '2': {'mcq': 'Which stage of RAG involves creating a data structure for querying the data?',
  'options': {'a': 'Loading',
   'b': 'Indexing',
   'c': 'Storing',
   'd': 'Querying'},
  'correct': 'b'},
 '3': {'mcq': 'What is the purpose of embeddings in RAG?',
  'options': {'a': 'To generate numerical representations of data',
   'b': 'To store metadata about the data',
   'c': 'To efficiently retrieve relevant context from an index',
   'd': 'To apply transformations to retrieved nodes'},
  'correct': 'a'},
 '4': {'mcq': 'What is the role of a router in RAG?',
  'options': {'a': 'To generate a response from an LLM',
   'b': 'To select the best re

In [36]:
for key, value in quiz.items():
    print(key, value)

1 {'mcq': 'What is the purpose of Retrieval-Augmented Generation (RAG)?', 'options': {'a': 'To train LLMs on enormous bodies of data', 'b': 'To add user data to the data LLMs already have access to', 'c': 'To create vector embeddings for data indexing', 'd': 'To evaluate the effectiveness of LLM responses'}, 'correct': 'b'}
2 {'mcq': 'Which stage of RAG involves creating a data structure for querying the data?', 'options': {'a': 'Loading', 'b': 'Indexing', 'c': 'Storing', 'd': 'Querying'}, 'correct': 'b'}
3 {'mcq': 'What is the purpose of embeddings in RAG?', 'options': {'a': 'To generate numerical representations of data', 'b': 'To store metadata about the data', 'c': 'To efficiently retrieve relevant context from an index', 'd': 'To apply transformations to retrieved nodes'}, 'correct': 'a'}
4 {'mcq': 'What is the role of a router in RAG?', 'options': {'a': 'To generate a response from an LLM', 'b': 'To select the best retriever based on metadata and query', 'c': 'To filter and re-ra

In [24]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option} : {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append(

       { "MCQ":mcq,
        "Choces":options,
        "Correct": correct
}
    )

In [25]:
quiz_table_data

[{'MCQ': 'What is the purpose of Retrieval-Augmented Generation (RAG)?',
  'Choces': 'a : To train LLMs on enormous bodies of data | b : To add user data to the data LLMs already have access to | c : To create vector embeddings for data indexing | d : To evaluate the effectiveness of LLM responses',
  'Correct': 'b'},
 {'MCQ': 'Which stage of RAG involves creating a data structure for querying the data?',
  'Choces': 'a : Loading | b : Indexing | c : Storing | d : Querying',
  'Correct': 'b'},
 {'MCQ': 'What is the purpose of embeddings in RAG?',
  'Choces': 'a : To generate numerical representations of data | b : To store metadata about the data | c : To efficiently retrieve relevant context from an index | d : To apply transformations to retrieved nodes',
  'Correct': 'a'},
 {'MCQ': 'What is the role of a router in RAG?',
  'Choces': 'a : To generate a response from an LLM | b : To select the best retriever based on metadata and query | c : To filter and re-rank retrieved nodes | d :

In [26]:
quiz_df = pd.DataFrame(quiz_table_data)

In [27]:
quiz_df

,MCQ,Choces,Correct
0,What is the purpose of Retrieval-Augmented Gen...,a : To train LLMs on enormous bodies of data |...,b
1,Which stage of RAG involves creating a data st...,a : Loading | b : Indexing | c : Storing | d :...,b
2,What is the purpose of embeddings in RAG?,a : To generate numerical representations of d...,a
3,What is the role of a router in RAG?,a : To generate a response from an LLM | b : T...,b
4,Which category of data-backed LLM applications...,a : Query Engines | b : Chat Engines | c : Age...,b


In [28]:
quiz_df.to_csv("RAG.csv")

In [29]:
import logging
import os
from datetime import datetime

In [30]:
f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

'12_31_2023_18_38_19.log'